In [1]:
import psycopg2
import sys
import pprint
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import time , datetime , date

import plotly.plotly as py
import plotly.graph_objs as go
import plotly.offline as off


In [2]:
def postgre_connect(host,db):
    conn_string = "host='" + host + "' dbname='"+db+"' user='postgres' password='postgres'"
    print ("Connecting to database\n	->%s" % (conn_string))
    conn = psycopg2.connect(conn_string)
    cursor = conn.cursor()
    print ("Connected!\n")
    return (conn , cursor)

In [79]:
def get_frame(table):
    df = pd.read_sql("select * from " + table + " order by entry_id desc limit 1000000 " ,conn)
    print ('Leidos:',len(df))
    return df

def clean_data(frame):
    df2 = frame.copy()
    df2['datetime'] = df2.apply(lambda x : datetime.combine( x['event_date'] , x['event_time'] ) , axis = 1)
    df2['event_type'] = df2['event_type'].apply(lambda x : x.strip())
    df2['event_time'] = df2['event_time'].apply(lambda x : time.strftime(x , '%H:%M:%S.%f'))
    df2['event_date'] = df2['event_date'].apply(lambda x : date.strftime(x , '%Y-%m-%d'))
    df2 = df2.set_index(['datetime'])
    return df2


In [80]:
(conn , cursor ) = postgre_connect('nitsuga.com.ar','gtrader')

Connecting to database
	->host='nitsuga.com.ar' dbname='gtrader' user='postgres' password='postgres'
Connected!



In [81]:
df_raw_bitfinex_btcusd = get_frame('bitfinex_btcusd')
df_raw_bitfinex_xrpbtc = get_frame('bitfinex_xrpbtc')
df_raw_bitfinex_xrpusd = get_frame('bitfinex_xrpusd')

Leidos: 1000000
Leidos: 1000000
Leidos: 1000000


## Limpiamos los datos

In [32]:
df_raw_bitfinex_btcusd.head()

,entry_id,event_date,event_time,event_type,event_price,event_size,ask_price,ask_size,bid_price,bid_size
0,3997351,2018-02-02,23:11:36.778465,ASK,8691.0,1,8691.0,1,8690.1,1
1,3997350,2018-02-02,23:11:36.545496,ASK,8692.8,1,8692.8,1,8690.1,1
2,3997349,2018-02-02,23:11:36.537689,BID,8690.1,1,8692.8,1,8690.1,1
3,3997348,2018-02-02,23:11:35.748827,ASK,8692.8,1,8692.8,1,8690.2,1
4,3997347,2018-02-02,23:11:35.737129,ASK,8692.9,1,8692.9,1,8690.2,1


In [82]:
df_raw_bitfinex_btcusd_clean = clean_data(df_raw_bitfinex_btcusd)
df_raw_bitfinex_xrpbtc_clean = clean_data(df_raw_bitfinex_xrpbtc)
df_raw_bitfinex_xrpusd_clean = clean_data(df_raw_bitfinex_xrpusd)

## Ordenamos los datos en funcion del entry_id

In [83]:
df_raw_bitfinex_btcusd_clean = df_raw_bitfinex_btcusd_clean.sort_values(by=['entry_id'], ascending=True)

In [84]:
df_raw_bitfinex_xrpbtc_clean = df_raw_bitfinex_xrpbtc_clean.sort_values(by=['entry_id'], ascending=True)

In [85]:
df_raw_bitfinex_xrpusd_clean = df_raw_bitfinex_xrpusd_clean.sort_values(by=['entry_id'], ascending=True)

In [94]:
df_raw_bitfinex_xrpusd_clean.head()

,entry_id,event_date,event_time,event_type,event_price,event_size,ask_price,ask_size,bid_price,bid_size
datetime,,,,,,,,,,
2018-01-29 22:20:36.790489,3081145,2018-01-29,22:20:36.790489,BID,1.2456,2,1.246,1,1.2456,2
2018-01-29 22:20:36.800538,3081146,2018-01-29,22:20:36.800538,ASK,1.2460,1,1.246,1,1.2456,2
2018-01-29 22:20:36.809092,3081147,2018-01-29,22:20:36.809092,BID,1.2453,1,1.246,1,1.2453,1
2018-01-29 22:20:36.817452,3081148,2018-01-29,22:20:36.817452,BID,1.2450,9,1.246,1,1.2450,9
2018-01-29 22:20:36.825715,3081149,2018-01-29,22:20:36.825715,BID,1.2449,1,1.246,1,1.2449,1


In [90]:

def arbitraje_btc_xrp(askbtc,askxrp,bidxrp,q,comm):
    return (q/(askbtc * (1 + comm))/(askxrp*(1+comm))*(bidxrp*(1-comm)))
    

In [95]:
def get_most_recent(datetime1 , frame):
    if len(frame[frame.index < datetime1]) > 0:
        return frame[frame.index < datetime1].tail(1)
    else:
        return frame.tail(1)
    

def check_window(frame1 , frame2 , frame3):
    list_profit = []
    df_profit = pd.DataFrame()
    datetimes = frame1.index.values
    event_dates = frame1['event_date'].values
    event_times = frame1['event_time'].values
    event_types = frame1['event_type'].values
    event_prices = frame1['event_price'].values
    event_sizes = frame1['event_size'].values
    ask_prices = frame1['ask_price'].values
    ask_sizes = frame1['ask_size'].values
    bid_prices = frame1['bid_price'].values
    bid_sizes = frame1['bid_size'].values

    for i in range(0,len(datetimes)):
        
        ## btc/usd
        current_datetime_1 = datetimes[i]

        event_date = event_dates[i]
        event_time = event_times[i]
        event_type = event_types[i]
        event_price = event_prices[i]
        event_size = event_sizes[i]
        ask_price = ask_prices[i]
        ask_size = ask_sizes[i]
        bid_price = bid_prices[i]
        bid_size = bid_sizes[i]
        
        ## xrp/btc
        most_recent_frame_2 = get_most_recent(current_datetime_1 , frame2)
        
        frame2_ask_price = most_recent_frame_2.ask_price.values[0]
        frame2_ask_size = most_recent_frame_2.ask_size.values[0]
        
        ## xrp/usd
        most_recent_frame_3 = get_most_recent(current_datetime_1 , frame3)
        
        frame3_bid_price = most_recent_frame_3.bid_price.values[0]
        frame3_bid_size = most_recent_frame_3.bid_size.values[0]
        
        profit = arbitraje_btc_xrp(ask_price,frame2_ask_price,frame3_bid_price,100,0.002)
        list_profit.append(profit)
    df_profit["profit"] = list_profit
    return df_profit
    

In [96]:
df_test = check_window(df_raw_bitfinex_btcusd_clean, df_raw_bitfinex_xrpbtc_clean, df_raw_bitfinex_xrpusd_clean)

KeyboardInterrupt: 

In [ ]:
df_test.sort_values(by="profit",ascending=False)